In [3]:
import numpy as np
import mne

In [4]:
from mne.io import read_raw_gdf
from pathlib import Path
import numpy as np

root = Path("C:/Users/paull/Documents/GIT/BCI_MsC/notebooks/BCI_Comp_IV_2a/BCICIV_2a_gdf")

dataset_folder = root
mat_files = list(dataset_folder.iterdir())



In [5]:
dataset_dict = {
    "train": ["A02", "A07", "A09", "A01"],
    "validation": ["A03", "A06"],
    "test": ["A04", "A05"],
}
all_subjects = [f"A0{i}" for i in range(10)]

dataset_dict = {
    "train": ["A02"],
#     "train": ["A02", "A07"],
#     "validation": ["A03"],
#     "test": ["A04"],
}

In [6]:
from ica_benchmark.scoring import mutual_information, coherence, correntropy, apply_pairwise, apply_pairwise_parallel, SCORING_FN_DICT
from ica_benchmark.processing.ica import get_ica_transformers
import time
from ica_benchmark.io.load import join_gdfs_to_numpy, load_subjects_data



N_RUNS = 3

results = {}

fn_dict = {
    "MI": mutual_information,
    "correntropy": correntropy,
    "coherence": coherence
}

n_components_list = [4, 8, 12, 16, 20, 22]

try:
    datasets
except:
    datasets = load_subjects_data(root, datasets=dataset_dict, mode="both")

score_calculated_before = {}

for n_components in n_components_list:
    for ica_method, ica_transform in get_ica_transformers(n_components=n_components).items():
        for run_n in range(N_RUNS):
            
            gdfs = [datasets["train"][subject]["gdf"] for subject in datasets["train"]]
            joined_eeg, labels = join_gdfs_to_numpy(gdfs)

            ica_transform.fit(joined_eeg)
            

#             for dataset_name in ("test", "validation", "train"):
            for dataset_name in ("train",):

                for subject_id in datasets[dataset_name]:
                    

                    gdf_data = datasets[dataset_name][subject_id]["gdf"]   
                    data_before = gdf_data.get_data().T
                    
                    data_after = ica_transform.transform(data_before)

                    for fn_name in fn_dict:

                        print((fn_name, ica_method, dataset_name, subject_id, run_n, n_components))
                        
                        if (n_components > 5) or len(data_after) > 2e6:
                            apply_fn = apply_pairwise_parallel
                        else:
                            apply_fn = apply_pairwise
                        
                        if not (subject_id, fn_name) in score_calculated_before:
                            score_before = apply_pairwise_parallel(data_before, fn_dict[fn_name])
                            score_calculated_before[(subject_id, fn_name)] = score_before

                        start = time.time()
                        score_after = apply_fn(data_after, fn_dict[fn_name])
                        duration = time.time() - start
                        
                        results[(fn_name, ica_method, dataset_name, subject_id, run_n, n_components)] = {
                            "score_before": score_calculated_before[(subject_id, fn_name)],
                            "score_after": score_after,
                            "time": duration
                        }

In [8]:
def run_ica_experiment(root, dataset_dict, n_components, ica_method, dataset_name, subject_id, fn_name):
    datasets = load_subjects_data(root, datasets=dataset_dict, mode="both")


    gdfs = [datasets["train"][subject]["gdf"] for subject in datasets["train"]]
    
    joined_dataset = join_gdfs_to_numpy(gdfs)

    joined_eeg, labels = join_gdfs_to_numpy(gdfs)
    ica_transform = get_ica_transformers()[ica_method]
    ica_transform.fit(joined_eeg)

    gdf_data = datasets[dataset_name][subject_id]["gdf"]   
    data = gdf_data.get_data().T

    data_after = ica_transform.transform(data)

    if (n_components > 5) or len(data_after) > 2e6:
        apply_fn = apply_pairwise_parallel
    else:
        apply_fn = apply_pairwise

    start = time.time()
    score_after = apply_fn(data_after, SCORING_FN_DICT[fn_name])
    duration = time.time() - start

    return score_after, duration

run_ica_experiment(root, dataset_dict, 20, "fastica", "train", "A02", "coherence")

(31.26619024074708, 160.57177305221558)

In [9]:
datasets["train"]

NameError: name 'datasets' is not defined

In [ ]:
import pandas as pd

df = []
cols = ["scoring", "algorithm", "dataset", "subject_id", "run", "n_components", "score_before", "score_after", "time"]

for k, v in results.items():
    df.append(list(k) + list(v.values()))
pd.DataFrame(df, columns=cols).to_csv("results.csv")

In [ ]:
df = pd.DataFrame(df, columns=cols)
df.groupby(["scoring", "algorithm", "dataset", "subject_id", "n_components"]).mean().query(""" (dataset == "test") """)